# K-Cores


In this notebook, we will use cuGraph to identify the K-Cores clusters in a test graph  

Notebook Credits
* Original Authors: Bradley Rees
* Created:   10/28/2019
* Last Edit: 01/23/2020

RAPIDS Versions: 0.12.0a

Test Hardware
* GV100 32G, CUDA 10.0



## Introduction

Compute the k-core of the graph G based on the out degree of its nodes. A k-core of a graph is a maximal subgraph that contains nodes of degree k or more. This call does not support a graph with self-loops and parallel edges.

For a detailed description of the algorithm see: https://en.wikipedia.org/wiki/Degeneracy_(graph_theory)


To compute the K-Core cluster in cuGraph use: <br>
* __gc = cugraph.k_core(G, G, k=None, core_number=None)__
    * G: A cugraph.Graph object
    * k: optional, The _k_ value to use
    * core_number: optional, Precomputed core number of the nodes of the graph G.  The see noptebook on Core-Number
    
Returns:
* __gc__
    * A new graph the contains the K-Core

All vertices with the same partition ID are in the same cluster



## cuGraph Notice 
The current version of cuGraph has some limitations:

* Vertex IDs need to be 32-bit integers.
* Vertex IDs are expected to be contiguous integers starting from 0.

cuGraph provides the renumber function to mitigate this problem. Input vertex IDs for the renumber function can be either 32-bit or 64-bit integers, can be non-contiguous, and can start from an arbitrary number. The renumber function maps the provided input vertex IDs to 32-bit contiguous integers starting from 0. cuGraph still requires the renumbered vertex IDs to be representable in 32-bit integers. These limitations are being addressed and will be fixed soon.    

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)


### Prep

In [14]:
# Import needed libraries
import cugraph
import cudf

### Read data using cuDF

In [15]:
# Test file    
datafile='../data//karate-data.csv'

In [16]:
# read the data using cuDF
gdf = cudf.read_csv(datafile, delimiter='\t', names=['src', 'dst'], dtype=['int32', 'int32'] )

In [17]:
# create a Graph 
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src', destination='dst')

In [18]:
print("Main Graph")
print("\tNumber of Vertices: " + str(G.number_of_vertices()))
print("\tNumber of Edges:    " + str(G.number_of_edges()))

Main Graph
	Number of Vertices: 34
	Number of Edges:    156


### Now run K-Cores

In [19]:
# Call k-cores on the graph
kcg = cugraph.k_core(G) 

In [20]:
print("K-Core Graph")
print("\tNumber of Vertices: " + str(kcg.number_of_vertices()))
print("\tNumber of Edges:    " + str(kcg.number_of_edges()))

K-Core Graph
	Number of Vertices: 34
	Number of Edges:    50


### Let's looks at the results
The results show that the roughly 2/3s of the edges have been removed.  
Let's look at the degrees of the vertices

In [21]:
d = kcg.degrees()
d.sort_values(by='out_degree', ascending=False)

,vertex,in_degree,out_degree
2,3,7,7
0,1,6,6
1,2,6,6
3,4,5,5
8,9,5,5
13,14,5,5
7,8,4,4
30,31,4,4
32,33,4,4
33,34,4,4


It is clear that only vertices with more than out_degree of 4 are connected.  
We could remove all the disconnected vertices, or do a subgraph extraction with vertices where out_degree != 0

![Karate Club](../img/karete-kcore.jpg)

In [22]:
# We can also just get a list of all the remaining edges as COO
coo = kcg.view_edge_list()

In [23]:
# print out edge list
coo

,src,dst
0,1,2
1,1,3
2,1,4
3,1,8
4,1,9
5,1,14
6,2,1
7,2,3
8,2,4
9,2,8


### Just for fun
Let's try specifying a K value.  Looking at the original network picture, it is easy to see that most vertices has at least degree two.  
If we specify k = 2 then only one vertex should be dropped 

In [24]:
# Call k-cores on the graph
kcg2 = cugraph.k_core(G, k=2) 

In [25]:
print("K-Core Graph")
print("\tNumber of Vertices: " + str(kcg2.number_of_vertices()))
print("\tNumber of Edges:    " + str(kcg2.number_of_edges()))

K-Core Graph
	Number of Vertices: 34
	Number of Edges:    154


In [26]:
d2 = kcg2.degrees()
d2.query('out_degree == 0')

,vertex,in_degree,out_degree
11,12,0,0


___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___